In [1]:
import tensorflow as tf
import pandas as pd
import os
import time


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


###para la inferencia podemos o espera un intervalod e tiempo para crear un eventcount_vector o sera que creamos una pequena memoria con cierta cantidad de lieas de logs hacia atras y vamos creando vectores y probando.

Creo que no tiene sentido hacer inferencia de linea en linea
 
- TODO: pensar y validar en codigo como hacer la inferencia de vainas nuevas
 

- definir cuales son los indicadores que se mostraran a dayco, tiempo de procesamiento, memoria, eficiencia delproceso

- justificar con argumentos porque el proceso funcionara con la data dayco igual que con la data de prueba

- como hacer en tiempo de inferencia, podemos simular que un archivo se va llenando de logs???revisar como cambia un archivo de logs en el logparser???

In [2]:
'''
!apt-get update
!apt-get install -y wget bzip2
!apt-get install -y gcc perl git
!rm -rf /var/lib/apt/lists/*

!cd /
!mkdir anaconda
!cd anaconda
!wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh
!bash Anaconda2-5.2.0-Linux-x86_64.sh \n\n\n\n\n\n\n\n\n
!source ~/.bashrc
!cd ..
!rm -r anaconda 
'''

'\n!apt-get update\n!apt-get install -y wget bzip2\n!apt-get install -y gcc perl git\n!rm -rf /var/lib/apt/lists/*\n\n!cd /\n!mkdir anaconda\n!cd anaconda\n!wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh\n!bash Anaconda2-5.2.0-Linux-x86_64.sh \n\n\n\n\n\n\n\n\n\n!source ~/.bashrc\n!cd ..\n!rm -r anaconda \n'

## clonamos loglizer

In [3]:
#!git clone https://github.com/elieser1101/loglizer.git /loglizer/

In [4]:
#os.chdir('/loglizer')
#!git pull origin master

## iniciamos submodulo de git

In [5]:
#os.chdir('/loglizer')
#!git submodule update --init

In [6]:
!pwd
!ls

/home/us1/git/loglizer
11_2_pipeline_deepia_intentto_online.ipynb  img		 rocore1.log
dayco_log.log				    __init__.py  rocore2.log
dayco_log.log_structured.csv		    LICENSE	 Test.log
dayco_log.logTemplateMap.csv		    logparser	 time_windows
dayco_log.log_templates.csv		    models	 utils
demo_bgl				    pipeline
demo_hdfs				    README.md


## importamos sys y agregamos algunas rutas necesarias para el logparser al path de python

In [7]:
import sys
#TODO: esto se debe incluir en repo, ver si hay una manera correcta o si es haciendo append del path!
#os.chdir('/loglizer')
repo_path = '/home/us1/git/loglizer'
sys.path.append(repo_path)#logsig __init__.py
sys.path.append(repo_path + '/logparser/logparser/LogSig')#logsig __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/')#for lenma __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/templateminer')#for lenma
from pipeline.pipeline import Pipeline
from pipeline.pipeline import Parser
from pipeline.pipeline import Loglizer
#os.chdir('/loglizer')

## mining invariants

### hack para probar el loglizer en pipeline,
luego de instanciar pipeline, cambiamos los atributos, para que apunten al dataset con el que funciona mining invariants, y creamos una nueva instancia de loglizer actualmente el `loglizer_input_dir` apunta a la salida del parser que es correcto, pero hasta que no se tenga el MiningInvariants adaptado a esa entrada se rompera
<br><br>
###NOTA: se puede jugar y obtener cualquier intancia de parser modificando los atributos de pipeline y haciendo get_parser, lo mismo para los loglizers

### visualizamos logs dayco

In [8]:
!head {repo_path}/rocore1.log
print('******')
!head {repo_path}/rocore2.log

Oct  4 14:30:04 200.74.216.1 3290: 009042: Oct  4 14:31:07.046 CCS: %SYS-6-LOGGINGHOST_STARTSTOP: Logging to host 192.168.1.140 port 0 CLI Request Triggered
Oct  4 14:30:05 200.74.216.1 3291: 009043: Oct  4 14:31:08.046 CCS: %SYS-6-LOGGINGHOST_STARTSTOP: Logging to host 192.168.1.140 port 514 started - CLI initiated
Oct  4 14:30:39 200.74.216.1 3292: 009044: Oct  4 14:31:41.731 CCS: %SYS-5-CONFIG_I: Configured from console by admbautistay on vty0 (200.74.215.2)
Oct  4 15:02:04 200.74.216.1 3293: 009052: Oct  4 15:03:07.365 CCS: %SYS-6-TTY_EXPIRE_TIMER: (exec timer expired, tty 1 (200.74.215.2)), user admbautistay
Oct  4 15:02:04 200.74.216.1 3294: 009053: Oct  4 15:03:07.365 CCS: %SYS-6-LOGOUT: User admbautistay has exited tty session 1(200.74.215.2)
Oct  4 21:01:43 200.74.216.1 3295: 009069: Oct  4 21:02:46.664 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:02:46 CCS Thu Oct 4 2018
Oct  4 21:01:46 200.74.216.1 3296: 009070: O

# Pipeline
## intentemos correr pipeline


### nota importante, es deinteres seleccionar con cuidado el parser pues, de la cantidad de eventos que este descubra dependera el gasto computacional del mining invarina


### seleccionaamos la cantidade de lineas con la que trabajaremos

In [9]:
#!cp {repo_path}/rocore2.log {repo_path}/dayco_log.log

#!wc -l {repo_path}/dayco_log.log
#!ls -sh {repo_path}/dayco_log.log

In [10]:
#solo par auna pruea
#!head -100 {repo_path}/rocore2.log > {repo_path}/dayco_log.log
!wc -l {repo_path}/dayco_log.log


15 /home/us1/git/loglizer/dayco_log.log


### NOTA: es importante que la ultima linea del archivo inicial con el que trabajeroms no contenga el salto de linea
nosotros simplemente estamos copiando la ultima linea sin el salto de linea
### NOTA:cuando se utilice un dataset completo tener cuidado de no darle el tratamiento que sigue

In [11]:

file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path+'/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()

In [12]:
print(len(lines))
print(lines[-2:])
!wc -l {repo_path}/dayco_log.log


16
['Oct  5 10:23:48 200.74.216.13 321: 000737: Oct  5 10:24:52.092 CCS: %DUAL-5-NBRCHANGE: EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is up: new adjacency\n', 'Oct  5 10:23:48 200.74.216.13 321: 000737: Oct  5 10:24:52.092 CCS: %DUAL-5-NBRCHANGE: EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is up: new adjacency']
15 /home/us1/git/loglizer/dayco_log.log


In [13]:
input_dir  = repo_path +'/'  # The input directory of log file
output_dir = repo_path + '/'  # The output directory of parsing results
log_file   = 'dayco_log.log'  # The input log file name
#log_format = '<Label> <Timestamp> <Date> <Node> <Time> <NodeRepeat> <Type> <Component> <Level> <Content>'#BGL
log_format = '<smonth> <sday> <shour> <ip> <id> <id2> <month> <day> <hour> <city> <type> <Content>'#dayco/rsyslog

pipeline = Pipeline(parser_algorithm='drain', input_dir = input_dir, parser_output_dir = output_dir, log_file = log_file,
parser_regex = log_format, feature_extractor='fixed_window', log_analizer_algorithm='mining_invariants',
data_type='time_based')

### en time_windows el feature extracto guarda una copia de lso indices de inicio y final de cada sliding window, por eso debemos borrar ese directorio cuando vamos a probar con archivos de diferentes tamanos(si se mantien la misma configuracion de swliding window y step size)

In [14]:
!rm /dayco_log.log_structured.csv
!rm /dayco_log.log_templates.csv
!rm /dayco_log.logTemplateMap.csv
!rm -r /time_windows

rm: cannot remove '/dayco_log.log_structured.csv': No such file or directory
rm: cannot remove '/dayco_log.log_templates.csv': No such file or directory
rm: cannot remove '/dayco_log.logTemplateMap.csv': No such file or directory
rm: cannot remove '/time_windows': No such file or directory


### Logica Online

parsing
- vamos parseando cada evento que entra???
- esperamos que entre un minimo de logs???
- parsemos cada cierto tiempo???
<br> ##############<br>

que hacemos con el event count matrix??
- cada cuanto la actualizamos??
- la guardamos en csv?
- validamos la longitud y en funcion de si cmabio iniciamos otros procesos?
<br> ##############<br>

calculo de invarianes(tomar ne cuent que en general se tarda bastate esto)
- cada cuanto debemos correr encontrar invariantes??
- si sabemos que es poca data podemos correrlo cada vez que ingresa un log nuevo??
- es practico buscar invariantes con mucha periodicidad??
<br> ##############<br>

debemos hacer inferencia con log que vayan entrando
- esperamos que se tengan un minimo de eventos para armar un event count vector??
- esperamos x unidades de tiempo??? 
- o con cad log probamoss??

++++++++++++++++++++++++++++++++++++++++<br>
se me ocurre crear un metodo fisrt_time_trining, que ejecute parsing y genere una primer event_count_matrix y encuentre los primeros invariantes

pedir que se evaluen todos estos enventos

crear un metodo depia daemon, 
1.
que evalue si el logfile.log incremento su tamano o si cambio la fecha del ultimo registro, calcular cual es la diferencia de lineas(guardar cual fue la ultima linea que se parseo), ejecutar parser sobre diferencia del log
2.
sobre la diferencia del log agregar nuevo event vector, tomar en cuenta el echo de que se utiliza sliding windows, como incluyo el evento en los logs anteriores?????? se puede/tiene que actualizar la matriz???o mejor se calcula una nueva???
3. 
definimos un trshol de cantidad de logs para reiniciar el calculo de invariantes??
o mas bien el treshold debe ser temporal???
POR ULTIMO, se puede evaluar la posibilidad de ir guardando en csv un registro de invariantes???es tendria sentido????(creo que no porque me parece que los invariantes calculados solo con nuevos vectores romperan cundo se comparen con vectores viejos, ahor los viejos romperan con los nuevos??)
4.
partiendo de 1 y 2 deberiamos tener los nuevos vectores y simplemente podemos hacer una infeencia con los invriantes que se tengan para el momento1234

## corremos el proceso de generar event_count_matrix, encontrar invariantes y detectar anomalias de data original
el pipeline es capaz de correr sin etiquetas, con los metodos deepia


In [15]:
!rm {repo_path}/time_windows/sliding_3h_1h.csv

In [16]:
#loglizer params for BGL
para = {
'path':repo_path + '/',                 # directory for input data
'log_file_name':'dayco_log.log',           # filename for log data file
'log_event_mapping':'dayco_log.logTemplateMap.csv',   # filename for log-event mapping. A list of event index, where each row represents a log
'save_path': './time_windows/',             # dir for saving sliding window data files to avoid splitting
#'select_column':[0,4],                      # select the corresponding columns (label and time) in the raw log file
'select_column':[0,1,2],                      # select the corresponding columns (label and time) in the raw log file, en caso de depia no nos intersa la columna label
'window_size':3, #3                           # time window (unit: hour)
'step_size': 1,                             # step size (unit: hour)
'epsilon':2.0,                              # threshold for the step of estimating invariant space
'threshold':0.97,                           # percentage of vector Xj in matrix satisfies the condition that |Xj*Vi|<epsilon
'scale_list':[1,2,3],					    # list used to sacle the theta of float into integer
'stop_invar_num':3                          # stop if the invariant length is larger than stop_invar_num. None if not given
}
start = time.time()
#event_count_matrix = pipeline.log_analizer.get_event_count_matrix(para)
#invar_dict = pipeline.log_analizer.find_invariants(para, event_count_matrix)
#predictions, anomalies = pipeline.log_analizer.get_anomalies(para, event_count_matrix, invar_dict)
pipeline.initial_go(para)
end = time.time()
print('total execution time in seconds?',end - start)

Parsing file: /home/us1/git/loglizer/dayco_log.log
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.044297]
The raw data shape is (16, 1) and label shape is (16, 1)
there are 44 instances (sliding windows) in this dataset

There are 6 log events
(44, 6)
SVD decomposition results (U,S,V) size:  (6, 6) (6,) (6, 6)
The size of invariants should be 5
the remaining features are:  [[0], [1], [2], [3], [4], [5]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
selected matrix columns are [0, 5]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
0 exits and discard the following vector: 
selected matrix columns are [2, 3]
0 exits and discard the following vector: 
selected 

In [17]:
pipeline.run_online(para, 600, 30, 10)

hacer inferencia 2018-12-02 10:04:09.313383 <<
old, new 16 15
hacer inferencia 2018-12-02 10:04:19.313384 <<
old, new 16 15
(44, 6)
SVD decomposition results (U,S,V) size:  (6, 6) (6,) (6, 6)
The size of invariants should be 5
the remaining features are:  [[0], [1], [2], [3], [4], [5]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
selected matrix columns are [0, 5]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
0 exits and discard the following vector: 
selected matrix columns are [2, 3]
0 exits and discard the following vector: 
selected matrix columns are [2, 4]
selected matrix columns are [2, 5]
A valid invariant is found and the corresponding columns are  [-2.  1.] [2, 5]

hacer inferencia 2018-12-02 10:06:09.350704 <<
old, new 16 15
hacer inferencia 2018-12-02 10:06:19.350705 <<
old, new 16 15
(44, 6)
SVD decomposition results (U,S,V) size:  (6, 6) (6,) (6, 6)
The size of invariants should be 5
the remaining features are:  [[0], [1], [2], [3], [4], [5]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
selected matrix columns are [0, 5]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
0 exits and discard the following vector: 
selected matrix columns are [2, 3]
0 exits and discard the following vector: 
selected matrix columns are [2, 4]
selected matrix columns are [2, 5]
A valid invariant is found and the corresponding columns are  [-2.  1.] [2, 5]

hacer inferencia 2018-12-02 10:08:09.350761 <<
old, new 16 20
####
#######
#########
############
logica de inferencia activa, cambio el log
Parsing file: /home/us1/git/loglizer/dayco_log.log
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.017192]
The raw data shape is (20, 1) and label shape is (20, 1)
there are 50 instances (sliding windows) in this dataset

There are 8 log events
log template path is /home/us1/git/loglizer/dayco_log.log_templates.csv
the mined 2 invariants are: {frozenset({2, 5}): array([-2.,  1.]), frozenset({4, 5}): array([-2.,  1.])}
[[2, 5], [4, 5]]
[[-2.0, 1.0], [-2.0, 1.0]]
*******8anomaly*****************
index, row 21 [1. 4. 1. 0. 0. 0. 0. 0.]
cols [2, 5] [-2.0, 1.0]

 invariant [2, 5]
>>> (exec timer expired, tty 1 (200.74.215.2)), user admer
start, end 2 8
len de eventos en window 1
8  Oct  4  11:20:42  200.74.216.13  313:  000683:  Oct  4  11:21:44.962  CCS:  %SYS-6-TTY_EXPIRE_TIMER:  (exec timer expired, tty 1 (200.74.215.2)), us...  d3

hacer inferencia 2018-12-02 10:08:39.367351 <<
old, new 20 20
hacer inferencia 2018-12-02 10:08:49.367352 <<
old, new 20 20
(50, 8)
SVD decomposition results (U,S,V) size:  (8, 8) (8,) (8, 8)
The size of invariants should be 7
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
selected matrix columns are [1, 6]
selected matrix columns are [1, 7]
0 exits and discard the following vector: 
selected matrix columns are [2, 3

hacer inferencia 2018-12-02 10:09:39.368233 <<
old, new 20 20
hacer inferencia 2018-12-02 10:09:49.368234 <<
old, new 20 20
(50, 8)
SVD decomposition results (U,S,V) size:  (8, 8) (8,) (8, 8)
The size of invariants should be 7
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
selected matrix columns are [1, 6]
selected matrix columns are [1, 7]
0 exits and discard the following vector: 
selected matrix columns are [2, 3

hacer inferencia 2018-12-02 10:10:39.368239 <<
old, new 20 20
hacer inferencia 2018-12-02 10:10:49.368240 <<
old, new 20 20
(50, 8)
SVD decomposition results (U,S,V) size:  (8, 8) (8,) (8, 8)
The size of invariants should be 7
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
selected matrix columns are [1, 2]
selected matrix columns are [1, 3]
0 exits and discard the following vector: 
selected matrix columns are [1, 4]
0 exits and discard the following vector: 
selected matrix columns are [1, 5]
selected matrix columns are [1, 6]
selected matrix columns are [1, 7]
0 exits and discard the following vector: 
selected matrix columns are [2, 3

hacer inferencia 2018-12-02 10:11:39.368245 <<
old, new 20 20
hacer inferencia 2018-12-02 10:11:49.368247 <<
old, new 20 20


KeyboardInterrupt: 

In [ ]:
pipeline.last_log_lines_len

In [ ]:
print(len(pipeline.invar_dict))
pipeline.invar_dict

In [ ]:
pipeline.event_count_matrix.shape

In [ ]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

In [ ]:
data_dft = pd.read_csv(repo_path + '/time_windows/sliding_3h_1h.csv', delimiter=r',', header=None) #, parse_dates = [1], date_parser=dateparse)
#data_dft = data_dft[0:100]
data_dft['diff'] = data_dft[1] - data_dft[0]
print(len(data_dft))
data_dft

### simulamos un cambio en el log, y al ejecutar pipeline.deepslash se valida el cambio, parsea,  genremaos eventcount matrix y hacemos inferenciacon los mismo invariantes ue se calcularon primero

vemos que priemro eran 27 eventos y luego 29
tambien vemos que primeor se detectan 10 anomalias y uego 14

In [ ]:
!head -150 {repo_path}/rocore2.log > {repo_path}/dayco_log.log

!wc -l {repo_path}/dayco_log.log
!ls -sh {repo_path}/dayco_log.log

file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path + '/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()

In [ ]:
pipeline.deepslash(para)

In [ ]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

In [ ]:
data_dft = pd.read_csv(repo_path + '/time_windows/sliding_3h_1h.csv', delimiter=r',', header=None) #, parse_dates = [1], date_parser=dateparse)
#data_dft = data_dft[0:100]
data_dft['diff'] = data_dft[1] - data_dft[0]
print(len(data_dft))
data_dft

In [ ]:
pipeline.find_invariants(para)
print(len(pipeline.invar_dict))
pipeline.invar_dict

###con la misma cantidad de lineas inferimos, ahora con los nuevos invariantes

In [ ]:
pipeline.get_anomalies(para)

In [ ]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

In [ ]:
!cp {repo_path}/rocore2.log {repo_path}/dayco_log.log

!wc -l {repo_path}/dayco_log.log
!ls -sh {repo_path}/dayco_log.log

file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path +'/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()

In [ ]:
pipeline.deepslash(para)

In [ ]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

In [ ]:
pipeline.find_invariants(para)
print(len(pipeline.invar_dict))
pipeline.invar_dict

In [ ]:
pipeline.event_count_matrix.shape

In [ ]:
pipeline.deepslash(para)
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

### visualizamos los archivos relevantes de este experimento

In [ ]:
!head {repo_path}/dayco_log.log 
!head {repo_path}/dayco_log.log_structured.csv  
!head {repo_path}/dayco_log.logTemplateMap.csv
!head {repo_path}/dayco_log.log_templates.csv

In [ ]:
from datetime import datetime, timedelta

In [ ]:
def run_online(online_period, invariant_window, inference_window):
  online_period = timedelta(seconds=online_period)
  invar_window = timedelta(seconds=invariant_window)
  inference_window = timedelta(seconds=inference_window)
  start_time = datetime.now()
  last_invar_t = las_inference_t = now = datetime.now()
  while now - start_time < online_period:
    if now - last_invar_t > invar_window:
      last_invar_t = now
      print('calcular invar', now)
    if now - las_inference_t > inference_window:
      las_inference_t = now
      print('hacer inferencia', now, '<<')
    #print(now)
    now = datetime.now()

run_online(20, 4, 1)

In [21]:
from utils.downloader import *

In [20]:
index = "filebeat-6.5.1-2018.12.02"
source_log = "/var/log/rocore2.log"
result_log_path = repo_path + "/Test.log"
create_file_from_elastic(index, source_log, result_log_path)

file = open(result_log_path, 'r')
lines = file.readlines()
file.close()
print(len(lines))     
print(lines)

20
['Oct  3 16:15:00 200.74.216.13 309: 000648: Oct  3 16:16:01.919 CCS: %SYS-6-LOGOUT: User admer has exited tty session 1(200.74.215.2)\n', 'Oct  3 21:02:03 200.74.216.13 311: 000657: Oct  3 21:03:05.102 CCS: %SYS-6-LOGOUT: User admcod has exited tty session 1(200.74.215.2)\n', 'Oct  4 12:51:24 200.74.216.13 315: 000687: Oct  4 12:52:26.817 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admbautistay] [Source: 200.74.215.2] [localport: 22] at 12:52:26 CCS Thu Oct 4 2018\n', 'Oct  3 15:44:56 200.74.216.13 307: 000646: Oct  3 15:45:58.187 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 15:45:58 CCS Wed Oct 3 2018\n', 'Oct  3 21:02:00 200.74.216.13 310: 000656: Oct  3 21:03:02.610 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:03:02 CCS Wed Oct 3 2018\n', 'Oct  4 11:20:42 200.74.216.13 314: 000684: Oct  4 11:21:44.962 CCS: %SYS-6-LOGOUT: User admer has exited tty sess

In [ ]:
pipeline.last_log_lines_len = 1

In [ ]:
pipeline.run_online(para, 600, 30, 10)

In [22]:
!head {repo_path}/dayco_log.log_structured.csv  


LineId,smonth,sday,shour,ip,id,id2,month,day,hour,city,type,Content,EventId,EventTemplate
1,Oct,3,16:15:00,200.74.216.13,309:,000648:,Oct,3,16:16:01.919,CCS:,%SYS-6-LOGOUT:,User admer has exited tty session 1(200.74.215.2),10b52bd2,User <*> has exited tty session 1(200.74.215.2)
2,Oct,3,21:02:03,200.74.216.13,311:,000657:,Oct,3,21:03:05.102,CCS:,%SYS-6-LOGOUT:,User admcod has exited tty session 1(200.74.215.2),10b52bd2,User <*> has exited tty session 1(200.74.215.2)
3,Oct,4,12:51:24,200.74.216.13,315:,000687:,Oct,4,12:52:26.817,CCS:,%SEC_LOGIN-5-LOGIN_SUCCESS:,Login Success [user: admbautistay] [Source: 200.74.215.2] [localport: 22] at 12:52:26 CCS Thu Oct 4 2018,0bee83ca,Login Success [user: <*> [Source: 200.74.215.2] [localport: 22] at <*> CCS <*> Oct <*> 2018
4,Oct,3,15:44:56,200.74.216.13,307:,000646:,Oct,3,15:45:58.187,CCS:,%SEC_LOGIN-5-LOGIN_SUCCESS:,Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 15:45:58 CCS Wed Oct 3 2018,0bee83ca,Login Success [user: <*>

In [149]:
class ParsedFile:
    def __init__(self, structure_logfile_path):
        self.parsed_logs_list = list()
        self.structure_logfile_path = structure_logfile_path
        self.structured_log_df = None
        self.load()
    
    def load(self):
        self.structured_log_df = pd.read_csv(self.structure_logfile_path)
        for index, row in self.structured_log_df.iterrows():
            log = ParsedLog(row.LineId,
                           row.smonth,
                           row.sday,
                           row.shour,
                           row.ip,
                           row.id,
                           row.id2,
                           row.month,
                           row.day,
                           row.hour,
                           row.city,
                           row.type,
                           row.Content,
                           row.EventId,
                           row.EventTemplate
                           )
            self.parsed_logs_list.append(log)  
#TODO: este parsed log dependera del tipoi de log, esta clase es hardcode dayco, pero deberia ser capaz de funcionar con cualquier log
class ParsedLog:
    def __init__(self, line_id, smonth, sday, shour, ip, id1, id2, month, day,
       hour, city, log_type, content, event_id, event_template):
        self.line_id = line_id
        self.smonth = smonth
        self.sday = sday
        self.shour = shour
        self.ip = ip
        self.id1 = id1
        self.id2 = id2
        self.month = month
        self.day = day
        self.hour = hour
        self.city = city
        self.log_type = log_type
        self.content = content
        self.event_id = event_id
        self.event_template = event_template
        self.timestamp = self.month + '-' + str(self.day) + '-' + self.hour
        self.syslog_timestamp = self.smonth + '-' + str(self.sday) + '-' + self.shour
        #matching attrs
        self.anomaly = False
        self.anomaly_vector_id = -1
        #TODO:agregar los capos para asociar con las anomallias, is_anomaly??habia otro??o solo con el objeto anomaly tambien puedo seleccinar las lineas
        
    def to_dict(self):
        return self.__dict__
        
class Indexer:
    def __init__(self, index_name, parsed_file):
        self.es = Elasticsearch()
        self.index_name = index_name
        self.parsed_file = parsed_file
    
    def index_log(self, log):
        response = self.es.index(index=self.index_name,
                                        doc_type='log',
                                        body=log,
                                        id=log['line_id'])
    def update_log(self, log, doc_id):
        response = self.es.update(index=self.index_name,
                                        doc_type='log',
                                        body=log,
                                        id=doc_id)
#TODO:que pasa cuando indexo un nuevo file???
#TODO:cuando ejecuto esto????
    def index_file(self):
        for parsed_log in self.parsed_file.parsed_logs_list:
            parsed_log_dict = parsed_log.to_dict()
            self.index_log(parsed_log_dict)
            
    def get_index(self):
        s = Search(using=self.es, index=self.index_name)
        res = s.scan()
        return res
    
    def delete_index(self, index_name):
        self.es.indices.delete(index=index_name)
        

In [150]:
parsed_file = ParsedFile(repo_path+'/dayco_log.log_structured.csv' )
for log in parsed_file.parsed_logs_list:
    print(log.content)

User admer has exited tty session 1(200.74.215.2)
User admcod has exited tty session 1(200.74.215.2)
Login Success [user: admbautistay] [Source: 200.74.215.2] [localport: 22] at 12:52:26 CCS Thu Oct 4 2018
Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 15:45:58 CCS Wed Oct 3 2018
Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:03:02 CCS Wed Oct 3 2018
User admer has exited tty session 1(200.74.215.2)
Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 10:35:25 CCS Thu Oct 4 2018
(exec timer expired, tty 1 (200.74.215.2)), user admer
Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:02:55 CCS Thu Oct 4 2018
User admbautistay has exited tty session 1(200.74.215.2)
Big chunk pool request (348) for community. Replenishing with malloc
EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
Configured from console by admer on vty0 (200.74.215.2)
(exec timer expired,

In [151]:
indexer = Indexer('deepia', parsed_file)

In [108]:
#indexer.delete_index('deepia')

In [153]:
indexer.index_file()

In [105]:
res = indexer.get_index()

In [106]:
a = list(res)
a

[<Hit(deepia/log/14): {'line_id': 14, 'smonth': 'Oct', 'sday': 3, 'shour': '16:15:...}>,
 <Hit(deepia/log/5): {'line_id': 5, 'smonth': 'Oct', 'sday': 3, 'shour': '21:02:0...}>,
 <Hit(deepia/log/2): {'line_id': 2, 'smonth': 'Oct', 'sday': 3, 'shour': '21:02:0...}>,
 <Hit(deepia/log/1): {'line_id': 1, 'smonth': 'Oct', 'sday': 3, 'shour': '16:15:0...}>,
 <Hit(deepia/log/3): {'line_id': 3, 'smonth': 'Oct', 'sday': 4, 'shour': '12:51:2...}>,
 <Hit(deepia/log/19): {'line_id': 19, 'smonth': 'Oct', 'sday': 5, 'shour': '16:40:...}>,
 <Hit(deepia/log/8): {'line_id': 8, 'smonth': 'Oct', 'sday': 4, 'shour': '11:20:4...}>,
 <Hit(deepia/log/4): {'line_id': 4, 'smonth': 'Oct', 'sday': 3, 'shour': '15:44:5...}>,
 <Hit(deepia/log/7): {'line_id': 7, 'smonth': 'Oct', 'sday': 4, 'shour': '10:34:2...}>,
 <Hit(deepia/log/11): {'line_id': 11, 'smonth': 'Oct', 'sday': 4, 'shour': '22:26:...}>,
 <Hit(deepia/log/9): {'line_id': 9, 'smonth': 'Oct', 'sday': 4, 'shour': '21:01:5...}>,
 <Hit(deepia/log/6): {'line_i

In [121]:
pipeline.anomalies

In [123]:
anomaly_events = list()
for i in pipeline.anomalies:
    for j in i.anomaly_events:
        anomaly_events.append(j)
anomaly_events

In [147]:
def set_doc_as_anomaly(anomaly_event):
    for anomaly_log_line_value in anomaly_event.log_lines_list:
        #print(anomaly_log_line_value.log_lines_list)
        update_elems = dict()
        update_elems['doc'] = dict()
        update_elems['doc']['anomaly'] = True
        update_elems['doc']['anomaly_vector_id'] = anomaly_event.row_index
        indexer.update_log(update_elems, anomaly_log_line_value)

In [154]:
for anomaly_event in anomaly_events:
    print(anomaly_event.log_lines_list)
    set_doc_as_anomaly(anomaly_event)
    

['8']
[]
['12']
[]
['14']
['13']
['18']
[]
